In [1]:
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LeakyReLU, Input, Flatten, MaxPooling1D, Conv1D, MaxPool1D, GlobalMaxPool1D, Dense, AveragePooling1D, GlobalAveragePooling1D

In [2]:
import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt
%matplotlib tk

In [3]:
import os, os.path, sys

In [4]:
path = r"C:\Users\mata2\Desktop\master\podaci\0k\Y"

In [5]:
dir = os.listdir(path)

Ucitavanje fajlova

In [6]:
file_names = []
for file in dir:
    file_names.append(file)

In [7]:
percent_train = 0.8

In [8]:
train_files_names = file_names[:int(percent_train*len(file_names))]
test_file_names = file_names[int(percent_train*len(file_names))::]

Ucitavanje zadate trajektorije

In [9]:
y_command = np.asarray(pd.read_csv(r"C:\Users\mata2\Desktop\master\podaci\yCmd.txt",header=None).astype('int'))

In [10]:
command_length = len(y_command)

Konkatenacija fajlova koji se posle pravilno rasporedjuju na adekvatne train/test skupove

In [11]:
train_ = np.concatenate([pd.read_csv(path + "/" + file, header=None)[:61000].astype('int')
                              for file in train_files_names], axis=0)
test_ = np.concatenate([pd.read_csv(path + "/" + file, header=None)[:61000].astype('int')
                              for file in test_file_names], axis=0)

Prethodno konkatenizovani podaci se adekvatno dele kako bi se izbeglo fiktivno stanje (stanje pri kraju rada i pocetku novog ako su konkatenizovani podaci)

In [12]:
for j in range(len(train_files_names)):
    exec(f"razlika_trening_{j} = []")
    for i in range(61000):
        exec(f"razlika_trening_{j}.append(y_command[i] - train_[j*61000+i])")
    exec(f'razlika_trening_{j} = np.asarray(razlika_trening_{j})')        

In [14]:
for j in range(len(test_file_names)):
    exec(f"razlika_test_{j} = []")
    for i in range(61000):
        exec(f"razlika_test_{j}.append(y_command[i] - test_[j*61000+i])")
    exec(f'razlika_test_{j} = np.asarray(razlika_test_{j})')    

Funkcija koja deli podatke na "sliding window"

In [13]:
def create_timeseries_split(data, time_steps):

    # Get the number of samples in the data
    num_samples = len(data) - time_steps

    # Create empty arrays to store features and target values
    x_train = np.zeros((num_samples, time_steps, 1))
    y_train = np.zeros((num_samples, 1))

    # Loop through the data and create features and target values
    for i in range(num_samples):
        # Extract a slice of data for the current feature
        x_train[i] = data[i:i+time_steps, :]

        # The target value is the next value after the feature
        y_train[i] = data[i+time_steps, 0]

    return x_train, y_train

Velicina buffera

In [15]:
time_steps = 10

Deljenje podataka na train i test skup

In [16]:
for i in range(len(train_files_names)):
    exec(f'train_X_{i}, train_y_{i} = create_timeseries_split(razlika_trening_{i}, time_steps)')

In [17]:
for i in range(len(test_file_names)):
    exec(f'test_X_{i}, test_y_{i} = create_timeseries_split(razlika_test_{i}, time_steps)')

In [18]:
train_X = np.asarray(train_X_0)
for i in range(1,len(train_files_names)):
    exec(f'train_X = np.append(train_X, train_X_{i}, axis=0)')

In [19]:
train_y = np.asarray(train_y_0)
for i in range(1,len(train_files_names)):
    exec(f'train_y = np.append(train_y, train_y_{i}, axis=0)')

In [20]:
test_X = np.asarray(test_X_0)
for i in range(1,len(test_file_names)):
    exec(f'test_X = np.append(test_X, test_X_{i}, axis=0)')

In [21]:
test_y = np.asarray(test_y_0)
for i in range(1,len(test_file_names)):
    exec(f'test_y = np.append(test_y, test_y_{i}, axis=0)')

In [22]:
input_shape = (time_steps, 1)
input_shape

(10, 1)

In [23]:
keras.utils.set_random_seed(7)
optimizerMSE = keras.optimizers.Adam(learning_rate=0.0001)

modelMSE = Sequential()
modelMSE.add(Input(shape=input_shape))
modelMSE.add(Conv1D(filters=16, kernel_size=3,padding='same', activation=LeakyReLU()))
modelMSE.add(MaxPooling1D(pool_size=4))
modelMSE.add(Flatten())
modelMSE.add(Dense(8, activation=LeakyReLU()))
modelMSE.add(Dense(units = 1, activation=LeakyReLU()))
modelMSE.compile(loss='mean_squared_error', optimizer = optimizerMSE)

In [24]:
historyMSE = modelMSE.fit(train_X, train_y, epochs=100, batch_size = 256, shuffle=False, verbose = 2)

Epoch 1/100

5718/5718 - 10s - loss: 20.3215 - 10s/epoch - 2ms/step
Epoch 2/100
5718/5718 - 10s - loss: 8.8391 - 10s/epoch - 2ms/step
Epoch 3/100
5718/5718 - 9s - loss: 7.5137 - 9s/epoch - 2ms/step
Epoch 4/100
5718/5718 - 9s - loss: 6.9288 - 9s/epoch - 2ms/step
Epoch 5/100
5718/5718 - 9s - loss: 6.4938 - 9s/epoch - 2ms/step
Epoch 6/100
5718/5718 - 9s - loss: 6.1281 - 9s/epoch - 2ms/step
Epoch 7/100
5718/5718 - 9s - loss: 5.7701 - 9s/epoch - 2ms/step
Epoch 8/100
5718/5718 - 9s - loss: 5.4557 - 9s/epoch - 2ms/step
Epoch 9/100
5718/5718 - 9s - loss: 5.2410 - 9s/epoch - 2ms/step
Epoch 10/100
5718/5718 - 9s - loss: 5.0886 - 9s/epoch - 2ms/step
Epoch 11/100
5718/5718 - 9s - loss: 4.9708 - 9s/epoch - 2ms/step
Epoch 12/100
5718/5718 - 9s - loss: 4.8766 - 9s/epoch - 2ms/step
Epoch 13/100
5718/5718 - 9s - loss: 4.7986 - 9s/epoch - 2ms/step
Epoch 14/100
5718/5718 - 9s - loss: 4.7295 - 9s/epoch - 2ms/step
Epoch 15/100
5718/5718 - 9s - loss: 4.6600 - 9s/epoch - 2ms/step
Epoch 16/100
5718/5718 - 9s 

In [26]:
train_scores = modelMSE.evaluate(test_X, test_y)

11436/11436 [==============================] - 12s 1ms/step - loss: 3.7042


In [26]:
#train_predict_MSE = modelMSE.predict(train_X)
test_predict_MSE = modelMSE.predict(test_X)

11436/11436 [==============================] - 18s 1ms/step


In [27]:
modelMSE.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 10, 16)            64        
                                                                 
 max_pooling1d (MaxPooling1  (None, 2, 16)             0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 8)                 264       
                                                                 
 dense_1 (Dense)             (None, 1)                 9         
                                                                 
Total params: 337 (1.32 KB)
Trainable params: 337 (1.32 KB)
Non-trainable params: 0 (0.00 Byte)
__________________________

Ucitavamo standardne devijacije i srednje vrednosti

In [23]:
std_dev = []
with open(r"C:\Users\mata2\Desktop\master\Axis-projection-RNN\odstupanja\0k Y\Y_std_dev.txt", 'r') as lines:
    for line in lines:
        std_dev.append(float(line.strip('\n')))

In [24]:
sr_vrednost = []
with open(r"C:\Users\mata2\Desktop\master\Axis-projection-RNN\odstupanja\0k Y\Y_sr_vrednost.txt", 'r') as lines:
    for line in lines:
        sr_vrednost.append(float(line.strip('\n')))

mean 3 sigma

In [30]:
i = time_steps
mean_3std = 0
for j in range(len(test_predict_MSE)):
    if i%command_length == 0:
        i = time_steps
    if sr_vrednost[i] - 3*std_dev[i] <= test_predict_MSE[j] <= sr_vrednost[i] + 3*std_dev[i]:
            mean_3std += 1
    i += 1
mean_3std/len(test_predict_MSE)

0.9862682406951959

pravljenje min/max koord, posle se koriste za plot

In [45]:
min_kord = [float('inf') for x in range(61000)]
max_kord = [float('-inf') for x in range(61000)]

for i in range(command_length):    
    for j in range(len(train_files_names)):
        exec(f'current_file = razlika_trening_{j}')
        if current_file[i][0] < min_kord[i]:
            min_kord[i] = current_file[i][0]
        if current_file[i][0] > max_kord[i]:
            max_kord[i] = current_file[i][0]

cuvamo/ucitavamo model

# NA TEST SKUPU - plotovi trajektorija

test_ = realizovana trajektorija

test_predict_MSE = predvidjena RAZLIKA koordinata od y_command

razlika_test_{} = realizovana RAZLIKA od test podataka od y_command - CELA TRAJEKTORIJA

test_y_{} = REALIZOVANA y koordinata od razlika_test_{}

Y_COMMAND - TEST_ = ODSTUPANJE/RAZLIKA_TEST_

Y_COMMAND[TIME_STEPS] + test_predict_MSE[0] = PRVU KOORDINATU TRAJEKTORIJE PREDVIDJENE VREDNOSTI

In [27]:
for j in range(len(test_file_names)):
    exec(f'predvidjena_trajektorija_{j} = []')
    for i in range(61000 - time_steps):
        exec(f'predvidjena_trajektorija_{j}.append(y_command[i + time_steps] + test_predict_MSE[j*(61000-time_steps) + i])')

In [37]:
plt.figure(figsize=(12,7))
plt.subplot(211)
plt.axvline(x = 17676, color = 'b', label = 'axvline - full height')
plt.plot(y_command[time_steps::],'r')
plt.plot(predvidjena_trajektorija_0,'b')
plt.plot(test_[time_steps:command_length], 'g')
plt.show()

plt.subplot(212)
plt.axvline(x = 17676, color = 'b', label = 'axvline - full height')
plt.plot(y_command[time_steps::],'r')
plt.plot(predvidjena_trajektorija_0,'b')
plt.plot(test_[time_steps:command_length], 'g')
plt.show()

In [38]:
plt.figure(figsize=(12,7))
plt.subplot(211)
plt.axvline(x = 24260, color = 'b', label = 'axvline - full height')
plt.plot(y_command[time_steps::],'r')
plt.plot(predvidjena_trajektorija_1,'b')
plt.plot(test_[time_steps:command_length], 'g')
plt.show()

plt.subplot(212)
plt.axvline(x = 24260, color = 'b', label = 'axvline - full height')
plt.plot(y_command[time_steps::],'r')
plt.plot(predvidjena_trajektorija_1,'b')
plt.plot(test_[time_steps:command_length], 'g')
plt.show()

In [39]:
plt.figure(figsize=(12,7))
plt.subplot(211)
plt.axvline(x = 27240, color = 'b', label = 'axvline - full height')
plt.plot(y_command[time_steps::],'r')
plt.plot(predvidjena_trajektorija_2,'b')
plt.plot(test_[time_steps:command_length], 'g')
plt.show()

plt.subplot(212)
plt.axvline(x = 27240, color = 'b', label = 'axvline - full height')
plt.plot(y_command[time_steps::],'r')
plt.plot(predvidjena_trajektorija_2,'b')
plt.plot(test_[time_steps:command_length], 'g')
plt.show()

In [40]:
plt.figure(figsize=(12,7))
plt.subplot(211)
plt.axvline(x = 8620, color = 'b', label = 'axvline - full height')
plt.plot(y_command[time_steps::],'r')
plt.plot(predvidjena_trajektorija_3,'b')
plt.plot(test_[time_steps:command_length], 'g')
plt.show()

plt.subplot(212)
plt.axvline(x = 8620, color = 'b', label = 'axvline - full height')
plt.plot(y_command[time_steps::],'r')
plt.plot(predvidjena_trajektorija_3,'b')
plt.plot(test_[time_steps:command_length], 'g')
plt.show()

In [43]:
plt.figure(figsize=(12,7))
plt.subplot(211)
plt.axvline(x = 43400, color = 'b', label = 'axvline - full height')
plt.plot(y_command[time_steps::],'r')
plt.plot(predvidjena_trajektorija_4,'b')
plt.plot(test_[time_steps:command_length], 'g')
plt.show()

plt.subplot(212)
plt.axvline(x = 43400, color = 'b', label = 'axvline - full height')
plt.plot(y_command[time_steps::],'r')
plt.plot(predvidjena_trajektorija_4,'b')
plt.plot(test_[time_steps:command_length], 'g')
plt.show()

In [42]:
plt.figure(figsize=(12,7))
plt.subplot(211)
plt.axvline(x = 55900, color = 'b', label = 'axvline - full height')
plt.plot(y_command[time_steps::],'r')
plt.plot(predvidjena_trajektorija_5,'b')
plt.plot(test_[time_steps:command_length], 'g')
plt.show()

plt.subplot(212)
plt.axvline(x = 55900, color = 'b', label = 'axvline - full height')
plt.plot(y_command[time_steps::],'r')
plt.plot(predvidjena_trajektorija_5,'b')
plt.plot(test_[time_steps:command_length], 'g')
plt.show()

# NA TEST SKUPU - min/max plotovi

MIN MAX TRAJEKTORIJE

In [46]:
min_trajektorija = []
max_trajektorija = []
for i in range(command_length):
    min_trajektorija.append(y_command[i] + min_kord[i])
    max_trajektorija.append(y_command[i] + max_kord[i])

In [53]:
i = time_steps
plt.figure(figsize=(12,7))
plt.subplot(211)
for j in range(len(predvidjena_trajektorija_0)):
    if i%61000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_0[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_0[j], color='black', marker='o', markersize=4)
    else:
        plt.plot(i, predvidjena_trajektorija_0[j], color='red',marker='o',markersize=4)
    i += 1
plt.plot(y_command, linewidth = 1)
plt.axvline(x = 12800, color = 'b', label = 'axvline - full height')

i = time_steps
plt.subplot(212)
for j in range(len(predvidjena_trajektorija_0)):
    if i%61000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_0[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_0[j], color='black', marker='o', markersize=4)
    else:
        plt.plot(i, predvidjena_trajektorija_0[j], color='red',marker='o',markersize=4)
    i += 1
plt.plot(y_command, linewidth = 1)
plt.axvline(x = 12800, color = 'b', label = 'axvline - full height')
plt.show()

In [58]:
i = time_steps
plt.figure(figsize=(12,7))
plt.subplot(211)
for j in range(len(predvidjena_trajektorija_1)):
    if i%61000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_1[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_1[j], color='black', marker='o', markersize=4)
    else:
        plt.plot(i, predvidjena_trajektorija_1[j], color='red',marker='o',markersize=4)
    i += 1
plt.plot(y_command, linewidth = 1)
plt.axvline(x = 45800, color = 'b', label = 'axvline - full height')

i = time_steps
plt.subplot(212)
for j in range(len(predvidjena_trajektorija_1)):
    if i%61000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_1[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_1[j], color='black', marker='o', markersize=4)
    else:
        plt.plot(i, predvidjena_trajektorija_1[j], color='red',marker='o',markersize=4)
    i += 1
plt.plot(y_command, linewidth = 1)
plt.axvline(x = 45800, color = 'b', label = 'axvline - full height')
plt.show()

In [60]:
i = time_steps
plt.figure(figsize=(12,7))
plt.subplot(211)
for j in range(len(predvidjena_trajektorija_2)):
    if i%61000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_2[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_2[j], color='black', marker='o', markersize=4)
    else:
        plt.plot(i, predvidjena_trajektorija_2[j], color='red',marker='o',markersize=4)
    i += 1
plt.plot(y_command, linewidth = 1)
plt.axvline(x = 13000, color = 'b', label = 'axvline - full height')

i = time_steps
plt.subplot(212)
for j in range(len(predvidjena_trajektorija_2)):
    if i%61000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_2[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_2[j], color='black', marker='o', markersize=4)
    else:
        plt.plot(i, predvidjena_trajektorija_2[j], color='red',marker='o',markersize=4)
    i += 1
plt.plot(y_command, linewidth = 1)
plt.axvline(x = 13000, color = 'b', label = 'axvline - full height')
plt.show()

In [62]:
i = time_steps
plt.figure(figsize=(12,7))
plt.subplot(211)
for j in range(len(predvidjena_trajektorija_3)):
    if i%61000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_3[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_3[j], color='black', marker='o', markersize=4)
    else:
        plt.plot(i, predvidjena_trajektorija_3[j], color='red',marker='o',markersize=4)
    i += 1
plt.plot(y_command, linewidth = 1)
plt.axvline(x = 26000, color = 'b', label = 'axvline - full height')

i = time_steps
plt.subplot(212)
for j in range(len(predvidjena_trajektorija_3)):
    if i%61000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_3[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_3[j], color='black', marker='o', markersize=4)
    else:
        plt.plot(i, predvidjena_trajektorija_3[j], color='red',marker='o',markersize=4)
    i += 1
plt.plot(y_command, linewidth = 1)
plt.axvline(x = 26000, color = 'b', label = 'axvline - full height')
plt.show()

In [64]:
i = time_steps
plt.figure(figsize=(12,7))
plt.subplot(211)
for j in range(len(predvidjena_trajektorija_4)):
    if i%61000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_4[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_4[j], color='black', marker='o', markersize=4)
    else:
        plt.plot(i, predvidjena_trajektorija_4[j], color='red',marker='o',markersize=4)
    i += 1
plt.plot(y_command, linewidth = 1)
plt.axvline(x = 6500, color = 'b', label = 'axvline - full height')

i = time_steps
plt.subplot(212)
for j in range(len(predvidjena_trajektorija_4)):
    if i%61000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_4[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_4[j], color='black', marker='o', markersize=4)
    else:
        plt.plot(i, predvidjena_trajektorija_4[j], color='red',marker='o',markersize=4)
    i += 1
plt.plot(y_command, linewidth = 1)
plt.axvline(x = 6500, color = 'b', label = 'axvline - full height')
plt.show()

In [66]:
i = time_steps
plt.figure(figsize=(12,7))
plt.subplot(211)
for j in range(len(predvidjena_trajektorija_5)):
    if i%61000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_5[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_5[j], color='black', marker='o', markersize=4)
    else:
        plt.plot(i, predvidjena_trajektorija_5[j], color='red',marker='o',markersize=4)
    i += 1
plt.plot(y_command, linewidth = 1)
plt.axvline(x = 39500, color = 'b', label = 'axvline - full height')

i = time_steps
plt.subplot(212)
for j in range(len(predvidjena_trajektorija_5)):
    if i%61000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_5[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_5[j], color='black', marker='o', markersize=4)
    else:
        plt.plot(i, predvidjena_trajektorija_5[j], color='red',marker='o',markersize=4)
    i += 1
plt.plot(y_command, linewidth = 1)
plt.axvline(x = 39500, color = 'b', label = 'axvline - full height')
plt.show()